# Разбор проблемы оптимизации

Этот блокнот фиксирует, почему перебор параметров давал одинаковый результат, и как это исправить.

## 1. Воспроизведение текущего поведения
Оптимизатор передаёт параметры в `SignalConfig.indicator_params` с верхним регистром (`KATR`, `PerATR` и т.д.).
Однако вспомогательная функция `build_indicator` фильтрует только ключи в нижнем регистре и в итоге отбрасывает все значения.

In [ ]:
from pprint import pprint

original_params = {
    'KATR': 5,
    'PerATR': 20,
    'SMA': 3,
    'MinRA': 0,
    'FlATR': 0,
    'FlHL': 0,
}

def buggy_build_indicator(params):
    param_mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL',
    }
    indicator_params = {}
    for param_key, param_value in params.items():
        if param_key in param_mapping:
            indicator_params[param_mapping[param_key]] = param_value
    return indicator_params

pprint(buggy_build_indicator(original_params))

Результат пустой – индикатор всегда строится с дефолтами, поэтому итоговая доходность не зависит от параметров.

## 2. Исправление
Нужно нормализовать имена параметров перед сопоставлением. Проще всего перевести ключ в нижний регистр, а затем взять значение из `param_mapping`.

In [ ]:
def fixed_indicator_params(params):
    param_mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL',
    }
    indicator_params = {}
    for param_key, param_value in params.items():
        normalized_key = param_key.lower()
        if normalized_key in param_mapping and param_value is not None:
            indicator_params[param_mapping[normalized_key]] = param_value
    return indicator_params

pprint(fixed_indicator_params(original_params))

Теперь все параметры проходят, и индикатор реально меняется при бэктесте.

## 3. Готовая функция для блокнота
Ниже финальная версия `build_indicator`, которую можно скопировать в рабочий ноутбук перед запуском оптимизации. Она:
- принимает параметры в любом регистре;
- отбрасывает `None`;
- выводит, какие значения реально ушли в расчёт (по желанию).

In [ ]:
from signals.signal_registry import SignalRegistry

def build_indicator(df, params, *, verbose=False):
    vm_ind = SignalRegistry.get('VolatilityMedian')
    param_mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL',
    }
    indicator_params = {}
    for key, value in (params or {}).items():
        normalized_key = key.lower()
        mapped_key = param_mapping.get(normalized_key)
        if mapped_key and value is not None:
            indicator_params[mapped_key] = value
    if verbose:
        print(f'Параметры, переданные в VolatilityMedian: {indicator_params}')
    return vm_ind.calculate(df, **indicator_params)

## 4. Следующие шаги
1. В исходном ноутбуке переопределить `build_indicator` этой версией (и, при желании, скорректировать `create_signal_config`, чтобы в `indicator_params` сразу попадали нижние регистры).
2. Перезапустить оптимизацию — теперь разные комбинации параметров должны давать разные метрики.
3. По итогам можно сохранить лучший набор параметров через `results_store.top_n()` или выгрузку CSV.